In [1]:
from contextlib import asynccontextmanager
import datetime
from http import HTTPStatus
from telegram import Update
from telegram.ext import Application, CommandHandler
from telegram.ext._contexttypes import ContextTypes
from fastapi import FastAPI, Request, Response
import os
import nest_asyncio
from database import User, get_db
from database.models import Tracker

nest_asyncio.apply()
nepal_tz = datetime.timezone(datetime.timedelta(hours=5, minutes=45))
ptb = (
    Application.builder()
    .updater(None)
    .token(os.getenv("TELEGRAM_BOT_TOKEN"))
    .read_timeout(7)
    .get_updates_read_timeout(42)
    .build()
)
def is_price_in_range(target: float, current: float, delta_percent: float) -> bool:
        delta = current * (delta_percent / 100)
        return current - delta <= target <= current + delta
def check_time_delta(last_alert_time: datetime, delta: int) -> bool:
    return (datetime.datetime.now(nepal_tz) - last_alert_time).seconds >= delta
    
async def check_trackers(context: ContextTypes.DEFAULT_TYPE):
    async with get_db() as db:
        trackers = db.execute(Tracker).all()
        for tracker in trackers:
            ltp = tracker.script.script_details.last_traded_price
            alert_price = tracker.price
            price_delta = (tracker.price_delta)/100
            last_alert_time = tracker.triggerd_at
            if check_time_delta(last_alert_time, 10) and is_price_in_range(alert_price, ltp, price_delta):
                alert_message = (
                        f"<b>Price Alert</b>\n\n"
                        f"<b>Script:</b> <code>{tracker.script.name}</code>\n"
                        f"<b>Target:</b> <code>{tracker.price:,.2f}</code>\n"
                        f"<b>Current:</b> <code>{ltp:,.2f}</code>\n"
                        f"<b>Time:</b> <code>{datetime.now(nepal_tz).strftime('%Y-%m-%d %H:%M:%S')}</code>"
                    )
                tracker.triggerd_at = datetime.datetime.now(nepal_tz)
                await ptb.bot.send_message(tracker.user.chat_id, alert_message, parse_mode="HTML")
                db.commit()
@asynccontextmanager
async def lifespan(_: FastAPI):
    await ptb.bot.setWebhook(os.getenv("WEBHOOK_URL"))
    async with ptb:
        await ptb.start()
        ptb.job_queue.run_repeating(
            check_trackers,
            interval=30,
            first=1,
            name="check_trackers",
        )
        yield
        await ptb.stop()

app = FastAPI(lifespan=lifespan)

@app.post("/")
async def process_update(request: Request):
    req = await request.json()
    print(req)
    update = Update.de_json(req, ptb.bot)
    await ptb.process_update(update)
    return Response(status_code=HTTPStatus.OK)

async def start(update, _: ContextTypes.DEFAULT_TYPE):
    """Send a message when the command /start is issued."""
    async with get_db() as db:
        user = db.execute(User).filter(User.chat_id == update.effective_user.id).first()
        if user:
            return await update.message.reply_text(f"You are already registered.")
        db.add(User(chat_id=update.effective_user.id, username=update.effective_user.first_name))
        db.commit()
        await update.message.reply_text(f"Welcome to NEPSE Price Tracker {update.effective_user.first_name}!")

ptb.add_handler(CommandHandler("start", start))

In [2]:
import uvicorn
uvicorn.run(app, port=8000)

INFO:     Started server process [220929]
INFO:     Waiting for application startup.


INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


{'update_id': 181506206, 'message': {'message_id': 42, 'from': {'id': 7511224313, 'is_bot': False, 'first_name': 'McLovin', 'language_code': 'en'}, 'chat': {'id': 7511224313, 'first_name': 'McLovin', 'type': 'private'}, 'date': 1739505225, 'text': '/start', 'entities': [{'offset': 0, 'length': 6, 'type': 'bot_command'}]}}
INFO:     91.108.5.53:0 - "POST / HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [220929]
